<a href="https://colab.research.google.com/github/lucianosilva-github/paradigmas/blob/main/PARADIGMAS_AULA_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LINGUAGENS E PARADIGMAS - AULA 04**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**



*   Entender o design pattern Visitor em Python
*   Entender e praticar com Visitor para percorrer a árvore sintática


In [ ]:
!pip install rply

**DESIGN PATTERN VISITOR**

O Visitor é um design pattern comportamental que permite adicionar novos comportamentos à hierarquia de classes existente sem alterar nenhum código existente.

<img src=https://www.oodesign.com/images/behavioral/visitor-pattern.png> </img>


Vamos implementar este design pattern para as classes abaixo:

In [ ]:
class Expr(object):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)


class Int(Expr):
    def __init__(self, value):
        self.value = value


class Add(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Mul(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right


In [ ]:
class Visitor(object):
    pass


class Eval(Visitor):
    def visit_int(self, i):
        return i.value

    def visit_add(self, a):
        return a.left.accept(self) + a.right.accept(self)

    def visit_mul(self, a):
        return a.left.accept(self) * a.right.accept(self)


class Print(Visitor):
    def visit_int(self, i):
        return i.value

    def visit_add(self, a):
        return '(+ {} {})'.format(a.left.accept(self), a.right.accept(self))

    def visit_mul(self, a):
        return '(* {} {})'.format(a.left.accept(self), a.right.accept(self))


In [ ]:
expr = Add(Add(Int(4), Int(3)), Mul(Int(10), Add(Int(1), Int(1))))
print(expr.accept(Print()))
print(expr.accept(Eval()))

**EXERCÍCIOS**

Implementar um visitor de impressão e outro de cálculo da expressão para a gramática abaixo:

\<expression\> ::= NUMBER

       | \<expression\> "+" \<expression\>

       | \<expression\> "-" \<expression\>

       | \<expression\> "*" \<expression\>

       | \<expression\> "/" \<expression\>

       | "(" <expression> ")"

In [6]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

In [7]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def accept(self, visitor):
        return visitor.visit_number(self)

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def accept(self, visitor):
        return visitor.visit_add(self)

class Sub(BinaryOp):
    def accept(self, visitor):
        return visitor.visit_sub(self)

class Mul(BinaryOp):
    def accept(self, visitor):
        return visitor.visit_mul(self)

class Div(BinaryOp):
    def accept(self, visitor):
        return visitor.visit_div(self)


In [8]:
class Eval(Visitor):

  def visit_number(self,number):
    return number.value
  
  def visit_add(self, add):
    return self.visit(add.left) + self.visit(add.right)
  
  def visit_sub(self, sub):
    return self.visit_sub(sub.left) - self.visit(sub.right)
  
  def visit_mul(self, mul):
    return self.visit_mul(mul.left) * self.visit(mul.right)
  
  def visit_div(self, div):
    return self.visit_div(div.left) / self.visit(div.right)

In [9]:
class Print(Visitor):

  def print_number(self,number):
    print(number.value)
  
  def print_add(self, add):
    add.left.accept(self)
    print('+')
    add.right.accept(self)
  
  def print_sub(self, sub):
    sub.left.accept(self)
    print('-')
    sub.right.accept(self)
    
  def print_mul(self, mul):  
    mul.left.accept(self)
    print('*')
    mul.right.accept(self)

  def print_div(self, div):
    div.left.accept(self)
    print('/')
    div.right.accept(self)

In [10]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [11]:
arvore=parser.parse(lexer.lex('1+2*3'))